In [7]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind  

/tmp/xpython_42/1357754178.py:3: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.stats import ttest_ind


In [8]:
#Similating t-tests comparing two samples with equal means and standard deviations
n_tests = 1000       # run 1000 fake experiments
n_per_group = 30     # each group has 30 samples, this is just a random number
mean1, mean2 = 0, 0  # both groups have the same true mean
sd = 1 

In [9]:
p_values = [] #creating an empty list to hold all my p vals
for ns in range(n_tests):
    g1 = np.random.normal(mean1, sd, n_per_group)  # 30 samples from N(0,1)
    g2 = np.random.normal(mean2, sd, n_per_group)  # another 30 samples
    ns, p = ttest_ind(g1, g2)  # run t-test
    p_values.append(p)        # save the p-value
p_values = np.array(p_values)

In [12]:
print("Raw p values", p_values)
print("Proportion of p values < 0.05 is", np.mean(p_values < 0.05))

Raw p values [3.28102357e-01 4.61701884e-01 9.89431578e-01 1.57329425e-01
 1.65419852e-01 9.14939819e-01 9.26903470e-01 9.09176735e-01
 1.45186599e-01 6.50578975e-01 2.95439594e-01 7.82556483e-01
 1.44019029e-01 6.73180849e-01 1.99271411e-01 8.92756458e-01
 5.22120216e-01 6.30044007e-02 1.86801557e-01 5.47769919e-02
 4.02001616e-01 6.55017655e-01 2.04822528e-02 8.12260304e-01
 3.57955279e-01 8.62352163e-01 9.07546220e-01 3.90459585e-01
 3.00816067e-01 1.09870045e-01 6.18070601e-01 4.30521582e-01
 8.12196958e-01 5.34619138e-01 9.84623166e-01 3.80653679e-01
 1.30621141e-01 9.67476078e-01 7.96339123e-02 3.38527634e-02
 8.10633065e-01 5.56198858e-01 6.18823612e-01 1.49690313e-01
 5.64252390e-01 8.21785079e-01 8.57070220e-01 8.30053742e-01
 4.75917230e-02 4.88121974e-02 7.18483498e-01 4.30383199e-01
 4.18981260e-01 6.62646950e-01 4.79691429e-01 3.55495874e-01
 7.57074719e-01 9.32103512e-01 3.70944761e-01 6.51507093e-01
 9.29927793e-01 2.07731501e-01 8.96438274e-01 9.88922223e-01
 1.38368101

In [13]:
#Bonferroni correction
alpha = 0.05                       # standard significance level
m = len(p_values)                  # total number of tests (1000 here)
bonf_thresh = alpha / m            # Bonferroni corrected cutoff, idk this is just what youre suppose to do to run a bonferroni
reject_bonf = p_values < bonf_thresh   # Boolean array

print("Bonferroni threshold:", bonf_thresh)
print("Number significant (Bonferroni):", np.sum(reject_bonf))


Bonferroni threshold: 5e-05
Number significant (Bonferroni): 0


In [15]:
# Benjamini–Hochberg correction 
# first going to sort the p-values from smallest to largest
sorted_idx = np.argsort(p_values)      # indices that would sort p-values
sorted_p = p_values[sorted_idx]        # sorted p-values from 

# compute BH thresholds for each rank i
ranks = np.arange(1, m+1)              # ranks 1...m
bh_thresholds = (ranks / m) * alpha    # thresholds for each of the ranks, which is threshold = i/m * alpha

# find the largest i where p_(i) <= threshold
below = sorted_p <= bh_thresholds
if np.any(below):                      # if at least one p passes
    max_i = np.max(np.where(below))    # index of last passing test
    bh_cutoff = sorted_p[max_i]        # BH cutoff value
else:
    bh_cutoff = 0                      # no p-values pass

reject_bh = p_values <= bh_cutoff

print("BH cutoff:", bh_cutoff)
print("Number significant (BH):", np.sum(reject_bh))


BH cutoff: 0
Number significant (BH): 0


In [17]:
meanA, meanB = 1, 2    # <-- change only this line to try bigger differences
sd = 1
n_tests = 1000
n_per_group = 30
alpha = 0.05

In [18]:
p_values = []
for _ in range(n_tests):
    g1 = np.random.normal(mean1, sd, n_per_group)  # group 1 around mean1
    g2 = np.random.normal(mean2, sd, n_per_group)  # group 2 around mean2
    _, p = ttest_ind(g1, g2)                       # two-sample t-test
    p_values.append(p)
p_values = np.array(p_values)

# Quick sanity check: how many raw p-values are < 0.05 (before correction)?
print("Raw p < 0.05 (no correction):", np.mean(p_values < 0.05))


Raw p < 0.05 (no correction): 0.05


In [21]:
# Bonferroni correction again
m = len(p_values)                 # number of tests (1000)
bonf_thresh = alpha / m           # Bonferroni cutoff (very strict)
reject_bonf = p_values < bonf_thresh
print("Bonferroni threshold:", bonf_thresh)
print("Number significant (Bonferroni):", np.sum(reject_bonf))

# Benjamini–Hochberg correction
# sort the p-values from smallest to largest
sorted_idx = np.argsort(p_values)
sorted_p = p_values[sorted_idx]

#build rank-based BH thresholds t_i = (i/m)*alpha
ranks = np.arange(1, m+1)
bh_thresholds = (ranks / m) * alpha

#find the largest rank i where p_(i) <= t_i
below = sorted_p <= bh_thresholds
if np.any(below):
    max_i = np.max(np.where(below))
    bh_cutoff = sorted_p[max_i]      # BH cutoff p-value
else:
    bh_cutoff = 0.0                   # no discoveries
reject_bh = p_values <= bh_cutoff     # mark all p <= cutoff as significant

print("BH cutoff:", bh_cutoff)
print("Number significant (BH):", np.sum(reject_bh))



Bonferroni threshold: 5e-05
Number significant (Bonferroni): 0
BH cutoff: 0.0
Number significant (BH): 0
